In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import os

In [ ]:
# load age key
root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
metadata_path = os.path.join(root, "metadata")
age_key_df = pd.read_csv(os.path.join(metadata_path, "age_key_df.csv"), index_col=0)

# remove rows and add variables
morph_experiments = ["20230830", "20230831", "20231207", "20231208"]
ref_experiments = ["20231110", "20240307", "20231206"]
# generate an absolute referemce
# eid_vec = np.asarray([snip[:12] for snip in age_key_df["snip_id"]])
# date_vec = np.asarray([snip[:8] for snip in age_key_df["snip_id"]])
# age_key_df["embryo_id"] = eid_vec
# age_key_df["experiment_date"] = date_vec
eid_vec = age_key_df["embryo_id"].to_numpy()
eid_index = np.unique(eid_vec)
age_key_df["abs_stage_hpf"] = np.nan

for eid in eid_index:
    abs_time_vec = age_key_df.loc[eid_vec==eid, "abs_time_hr"].to_numpy()
    pd_stage_vec = age_key_df.loc[eid_vec==eid, "inferred_stage_hpf_reg"].to_numpy()
    age_key_df.loc[eid_vec==eid, "abs_stage_hpf"] = pd_stage_vec[0] + abs_time_vec

    
age_key_df["experiment_date"] = age_key_df["experiment_date"].astype(str)
keep_flags = np.asarray([True if age_key_df.loc[i, "experiment_date"] not in morph_experiments
                         else False for i in age_key_df.index])


age_key_df = age_key_df.loc[keep_flags, :]

ref_flags = np.asarray([True if age_key_df.loc[i, "experiment_date"] in ref_experiments
                         else False for i in age_key_df.index])
age_key_df.head(5)

In [ ]:
px.histogram(age_key_df.loc[ref_flags], x="calc_stage_hpf")

In [ ]:
fig = px.scatter(age_key_df, x="calc_stage_hpf", y="inferred_stage_hpf_reg",opacity=0.25,
                 color="experiment_date", template="plotly")
fig.show()

In [ ]:
age_key_df_alt = pd.read_csv(os.path.join(metadata_path, "age_key_df.csv"), index_col=0)
age_key_df_alt = age_key_df_alt.loc[~keep_flags, :]
age_key_df_alt.tail(50)